<a href="https://colab.research.google.com/github/dfavenfre/financial-sentiment-classifier/blob/main/financial-sentiment-classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚛ Packages and Important Libraries ⚛
The imported libraries contain various modules that are helpful for model building in tensorflow and for data, financial_phrasebank dataset from Huggingface is downloaded. Important resources can be found down below. 

*   https://huggingface.co/datasets/financial_phrasebank (Financial PhraseBank Dataset)
*   https://medium.com/prosus-ai-tech-blog/finbert-financial-sentiment-analysis-with-bert-b277a3607101 (FinBERT Article)
*   https://medium.com/mlearning-ai/optimizing-deep-learning-models-with-pruning-a-practical-guide-163e990c02af (Model Pruning in TF)

Complete list of libraries and modules are; 
*  tensorflow.keras.layers,  tensorflow.keras.models, tensorflow.keras.utils, sklearn.metrics, sklearn.model_selection
*  nltk, string, re, random, io, pandas, numpy



paketleri yeniden düzenle, hatalar var

In [ ]:
# model packages
!pip install datasets
!pip install imbalanced-learn
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras import layers 
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.python.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Attention
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from keras.utils import pad_sequences
from helper_functions import create_tensorboard_callback
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers, models, utils
from keras.utils import to_categorical

# data preprocessing packages
import re
import string
import nltk
from nltk.corpus import stopwords
from keras.utils import to_categorical

# uploading data
import random
import re
import io
import pandas as pd
from google.colab import files
from datasets import load_dataset

# visualization
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-05-27 15:59:16--  https://raw.githubusercontent.com/mrdbourke/tensor

In [ ]:
# create a directory to save TensorBoard Logs
SAVE_DIR = "model_logs"

# Useful Functions ⛽



*   **text_cleaning(text)** : Clean and remove unnecessary string elements from each sentence, and return it back to the original sentence data
*   **calculate_results(y_true, y_pred)** :  Calculate various classification scores, including Accuracy, F1_score, Recall, and Precision. 

An addition to weight parameter for "precision_recall_fscore_support(y_true, y_pred, average="weighted")", since the target variables (labels) might be imbalanced, setting average parameter to "weighted" solves the imbalance issue for us.   



In [ ]:
def text_cleaning(text):

  """ Clean and remove unnecessary string elements from each sentence, and return it back to the original sentence data."""

  text = re.sub(r'@[A-Za-z0-9]+', '', text)     # removing @mentions
  text = re.sub(r'@[A-Za-zA-Z0-9]+', '', text)  # removing @mentions 
  text = re.sub(r'@[A-Za-z]+', '', text)        # removing @mentions
  text = re.sub(r'@[-)]+', '', text)            # removing @mentions
  text = re.sub(r'#', '', text )                # removing '#' sign
  text = re.sub(r'RT[\s]+', '', text)           # removing RT
  text = re.sub(r'https?\/\/\S+', '', text)     # removing the hyper link
  text = re.sub(r'&[a-z;]+', '', text)          # removing '&gt;'

  return text

def calculate_results(y_true, y_pred):

  """ Calculate various classification scores, including Accuracy, F1_score, Recall, and Precision."""

  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": round(model_accuracy,3),
                    "precision": round(model_precision,3),
                    "recall": round(model_recall,3),
                    "f1": round(model_f1,3)}
  return model_results

def pre_process(data):

  """ Alternative to text_cleaning(), does the same job"""

  # Convert all text to lowercase
  data = data.str.lower()

  # Remove numbers
  data = data.apply(lambda x: re.sub(r'\d+', '', x))

  # Remove punctuation
  punctuation = string.punctuation.replace("'", "")
  data = data.apply(lambda x: x.translate(str.maketrans('', '', punctuation)))

  # Remove $ 
  punctuation = string.punctuation.replace("$", "")
  data = data.apply(lambda x: x.translate(str.maketrans('', '', punctuation)))

  # Remove stop words
  nltk.download('stopwords')
  stop_words = set(stopwords.words('english'))
  data = data.apply(lambda x: " ".join(word for word in x.split() if word not in stop_words))

  return data

# Upload Financial_Phrasebank Data 🤗

In [ ]:
# load the data using load_dataset
dataset = load_dataset("financial_phrasebank","sentences_allagree",split="train")
# Convert the DatasetDict to a Pandas dataframe (named it after HuggingFace DataFrame)
text_df = pd.DataFrame(dataset)
print("Total Number of Sentences is: {}".format(len(text_df["sentence"])))

Generating train split:   0%|          | 0/2264 [00:00<?, ? examples/s]

Dataset financial_phrasebank downloaded and prepared to /root/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141. Subsequent calls will reuse this data.
Total Number of Sentences is: 2264


In [ ]:
# Display the dataframe
text_df["label"].value_counts(normalize=True)

1    0.614399
2    0.251767
0    0.133834
Name: label, dtype: float64

In [ ]:
# clean and split huggingface dataset into training and validation
sentences = pre_process(text_df["sentence"])
X_train_pre, X_test_pre, y_train_pre, y_test_pre = train_test_split(sentences,
                                                                    text_df["label"],
                                                                    test_size=0.1,
                                                                    random_state=42)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Upload Economic Sentiment text from Kaggle ⚓

In [ ]:
uploaded = files.upload()
text_df = pd.read_csv(io.BytesIO(uploaded['economic_sentiment_text.csv']))

In [ ]:
# clean and split economic sentiment dataset into training and validation
text_df["Sentiment"] = text_df["Sentiment"].replace({"neutral":0, "positive":1, "negative":2})
text_df["Sentiment"].value_counts(normalize=True)


In [ ]:
sentences = pre_process(text_df["Sentence"])
X_train_pre, X_test_pre, y_train_pre, y_test_pre = train_test_split(sentences,text_df["Sentiment"],test_size=0.1,random_state=42)

# Data Pre-processing with Tensorflow Tokenizer 📑

In [ ]:
# Initialize the tokenizer 
tokenizer = Tokenizer()

# Fit the tokenizer on the training data
tokenizer.fit_on_texts(X_train_pre)

# Convert the text data to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train_pre)
X_test_seq = tokenizer.texts_to_sequences(X_test_pre)

# Find max length
max_length = max(len(seq) for seq in X_train_seq)
print("The max length is: ", max_length)

# Pad the sequences to have the same length
X_train = pad_sequences(X_train_seq, max_length)
X_test = pad_sequences(X_test_seq, max_length)

# converting target variables into categorical values
y_train = to_categorical(y_train_pre, num_classes=3)
y_test = to_categorical(y_test_pre, num_classes=3)

# Balance the labels
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

The max length is:  36


# Baseline Model (Dense) ⚫

In [ ]:
# construct the model architecture
inputs = layers.Input(shape=(max_length,))
x = layers.Embedding(input_dim=10000, output_dim=256, input_length=max_length)(inputs)
x = layers.Dropout(0.5)(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(3, activation="softmax")(x)
model = models.Model(inputs, outputs)

# compile the model
model.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 36)]              0         
                                                                 
 embedding (Embedding)       (None, 36, 256)           2560000   
                                                                 
 dropout (Dropout)           (None, 36, 256)           0         
                                                                 
 global_average_pooling1d (G  (None, 256)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                             

In [ ]:
# train the model
model_history_hf = model.fit(X_resampled, y_resampled, 
          validation_data=(X_test, y_test),
          epochs=4,
          batch_size=1,
          callbacks=[create_tensorboard_callback(SAVE_DIR,"model_dense")])

Saving TensorBoard log files to: model_logs/model_dense/20230527-155945
Epoch 1/4
3738/3738 [==============================] - 46s 10ms/step - loss: 0.9047 - accuracy: 0.5372 - val_loss: 0.9953 - val_accuracy: 0.6079
Epoch 2/4
3738/3738 [==============================] - 17s 4ms/step - loss: 0.7673 - accuracy: 0.6089 - val_loss: 0.6293 - val_accuracy: 0.7401
Epoch 3/4
3738/3738 [==============================] - 18s 5ms/step - loss: 0.6692 - accuracy: 0.6873 - val_loss: 0.6014 - val_accuracy: 0.7269
Epoch 4/4
3738/3738 [==============================] - 17s 5ms/step - loss: 0.5967 - accuracy: 0.7207 - val_loss: 0.6596 - val_accuracy: 0.7489


In [ ]:
optimized = pd.DataFrame(model_history_hf.history) 
best_parameters = optimized.loc[optimized["loss"] > optimized["val_loss"]]
best_parameters.sort_values(by="val_accuracy", ascending=False).head()

,loss,accuracy,val_loss,val_accuracy
1,0.767324,0.608882,0.629258,0.740088
2,0.669200,0.687266,0.601377,0.726872


In [ ]:
# predict 
model_1_preds_probs = model.predict(X_test)
# convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_preds_probs))
# calculate model_1 results
model_1 = calculate_results(y_test, model_1_preds)

8/8 [==============================] - 0s 2ms/step


# Model 2 (LSTM) ⬛

In [ ]:
# construct the model architecture
inputs = layers.Input(shape=(max_length,))
x = layers.Embedding(input_dim=10000, output_dim=256, input_length=max_length)(inputs)
x = layers.Dropout(0.5)(x)
x = layers.LSTM(256, return_sequences=True)(x)
x = layers.Dropout(0.5)(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(3, activation="softmax")(x)

model_lstm = models.Model(inputs, outputs)

# compile the model
model_lstm.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model_lstm.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 36)]              0         
                                                                 
 embedding_3 (Embedding)     (None, 36, 256)           2560000   
                                                                 
 dropout_8 (Dropout)         (None, 36, 256)           0         
                                                                 
 lstm_1 (LSTM)               (None, 36, 256)           525312    
                                                                 
 dropout_9 (Dropout)         (None, 36, 256)           0         
                                                                 
 global_average_pooling1d_3   (None, 256)              0         
 (GlobalAveragePooling1D)                                        
                                                           

In [ ]:
# train the model
model_history_lstm = model_lstm.fit(X_resampled, y_resampled, 
          validation_data=(X_test, y_test),
          epochs=5,
          callbacks=[create_tensorboard_callback(SAVE_DIR,"model_lstm")])

Saving TensorBoard log files to: model_logs/model_lstm/20230527-162643
Epoch 1/5
117/117 [==============================] - 15s 109ms/step - loss: 0.9769 - accuracy: 0.4789 - val_loss: 0.7596 - val_accuracy: 0.7093
Epoch 2/5
117/117 [==============================] - 6s 50ms/step - loss: 0.7622 - accuracy: 0.6097 - val_loss: 0.6662 - val_accuracy: 0.6740
Epoch 3/5
117/117 [==============================] - 3s 24ms/step - loss: 0.6691 - accuracy: 0.6458 - val_loss: 0.6509 - val_accuracy: 0.7048
Epoch 4/5
117/117 [==============================] - 2s 14ms/step - loss: 0.6056 - accuracy: 0.6969 - val_loss: 0.6270 - val_accuracy: 0.7709
Epoch 5/5
117/117 [==============================] - 1s 12ms/step - loss: 0.5234 - accuracy: 0.7643 - val_loss: 0.5921 - val_accuracy: 0.7665


In [ ]:
# predict 
model_2_preds_probs = model_lstm.predict(X_test)
# convert model prediction probabilities to label format
model_2_preds = tf.squeeze(tf.round(model_2_preds_probs))
# calculate model_1 results
model_2 = calculate_results(y_test, model_2_preds)

8/8 [==============================] - 0s 3ms/step


# Model 3 (GRU) ⬛

In [ ]:
# construct the model architecture
inputs = layers.Input(shape=(max_length, ))
x = layers.Embedding(input_dim=10000, output_dim=128, input_length=max_length)(inputs)
x = layers.Dropout(0.5)(x)
x = layers.GRU(128,return_sequences=True)(x)
x = layers.Dropout(0.5)(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(3, activation="softmax")(x)
model_gru = models.Model(inputs, outputs)

# compile the model
model_gru.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])
model_gru.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 36)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 36, 128)           1280000   
                                                                 
 dropout_4 (Dropout)         (None, 36, 128)           0         
                                                                 
 gru (GRU)                   (None, 36, 128)           99072     
                                                                 
 dropout_5 (Dropout)         (None, 36, 128)           0         
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                           

In [ ]:
# train the model
model_history_gru = model_gru.fit(X_resampled, y_resampled, 
          validation_data=(X_test, y_test),
          epochs=5,
          callbacks=[create_tensorboard_callback(SAVE_DIR,"model_gru")])

Saving TensorBoard log files to: model_logs/model_gru/20230527-161144
Epoch 1/5
117/117 [==============================] - 19s 133ms/step - loss: 0.9955 - accuracy: 0.4666 - val_loss: 0.7336 - val_accuracy: 0.6608
Epoch 2/5
117/117 [==============================] - 8s 67ms/step - loss: 0.7796 - accuracy: 0.5835 - val_loss: 0.6529 - val_accuracy: 0.6652
Epoch 3/5
117/117 [==============================] - 4s 35ms/step - loss: 0.6864 - accuracy: 0.6327 - val_loss: 0.6489 - val_accuracy: 0.7048
Epoch 4/5
117/117 [==============================] - 1s 13ms/step - loss: 0.6220 - accuracy: 0.6739 - val_loss: 0.6596 - val_accuracy: 0.7137
Epoch 5/5
117/117 [==============================] - 2s 20ms/step - loss: 0.5617 - accuracy: 0.7218 - val_loss: 0.6308 - val_accuracy: 0.7621


In [ ]:
# predict 
model_3_preds_probs = model_gru.predict(X_test)
# convert model prediction probabilities to label format
model_3_preds = tf.squeeze(tf.round(model_3_preds_probs))
# calculate model_1 results
model_3 = calculate_results(y_test, model_3_preds)

8/8 [==============================] - 0s 3ms/step


# Model 4 (Bidirectional LSTM) ⬛

In [ ]:
# construct the model architecture
inputs = layers.Input(shape=(max_length,))
x = layers.Dropout(0.5)(x)
x = layers.Embedding(input_dim=10000, output_dim=256, input_length=max_length)(inputs)
x = layers.Dropout(0.5)(x)
x = layers.Bidirectional(layers.LSTM(128,return_sequences=True))(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64, "relu")(x)
outputs = layers.Dense(3, activation="softmax")(x)
model_bd = models.Model(inputs, outputs)

# compile the model
model_bd.compile(loss="categorical_crossentropy", 
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model_bd.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 36)]              0         
                                                                 
 embedding_2 (Embedding)     (None, 36, 256)           2560000   
                                                                 
 dropout_7 (Dropout)         (None, 36, 256)           0         
                                                                 
 bidirectional (Bidirectiona  (None, 36, 256)          394240    
 l)                                                              
                                                                 
 global_average_pooling1d_2   (None, 256)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_7 (Dense)             (None, 64)                1644

In [ ]:
# train the model
model_history_bd = model_bd.fit(X_resampled, y_resampled, 
          validation_data=(X_test, y_test),
          epochs=8,
          callbacks=[create_tensorboard_callback(SAVE_DIR,"model_bd")])

Saving TensorBoard log files to: model_logs/model_bd/20230527-161220
Epoch 1/8
117/117 [==============================] - 21s 142ms/step - loss: 0.9316 - accuracy: 0.4960 - val_loss: 0.6228 - val_accuracy: 0.7225
Epoch 2/8
117/117 [==============================] - 8s 66ms/step - loss: 0.7240 - accuracy: 0.6049 - val_loss: 0.8006 - val_accuracy: 0.6344
Epoch 3/8
117/117 [==============================] - 3s 24ms/step - loss: 0.6299 - accuracy: 0.6651 - val_loss: 0.7028 - val_accuracy: 0.7533
Epoch 4/8
117/117 [==============================] - 4s 31ms/step - loss: 0.5566 - accuracy: 0.7199 - val_loss: 0.6088 - val_accuracy: 0.7885
Epoch 5/8
117/117 [==============================] - 3s 25ms/step - loss: 0.4803 - accuracy: 0.7809 - val_loss: 0.7387 - val_accuracy: 0.7313
Epoch 6/8
117/117 [==============================] - 3s 27ms/step - loss: 0.4138 - accuracy: 0.8277 - val_loss: 0.7261 - val_accuracy: 0.7533
Epoch 7/8
117/117 [==============================] - 2s 13ms/step - loss: 0.3

In [ ]:
# predict 
model_4_preds_probs = model_bd.predict(X_test)
# convert model prediction probabilities to label format
model_4_preds = tf.squeeze(tf.round(model_4_preds_probs))
# calculate model_1 results
model_4 = calculate_results(y_test, model_4_preds)

8/8 [==============================] - 1s 4ms/step


# Feature Extraction #1 ⬛

**A pre-trained sentence encoder (embedding) -> Universal Sentence Encoder (USE)**

**Resources**
* If you'd want to deploy the model for mobile, a lite-version of any pre-trained model would do the work. If otherwise, stick with the large ones. 
* https://tfhub.dev/google/universal-sentence-encoder/4 

**Import Notes:** 
* Do not import tensorflow_hub outside the scope of hub.load(""). If run outside, for some unknown reason the packages throws an error.    
* leave the input_shape as a blank list, because it has a pre-defined input_shape that we have no idea of (probably it has an input shape of 512)

In [ ]:
import tensorflow_hub as hub
embedding_fn = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [ ]:
# Create a Keras Layer using the USE pre-trained layer from tensorflow-hub 
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # check the important notes
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE_encoder")

In [ ]:
# Run this only once, otherwise the shape of the label data would look like (0.9*len(y_train),3,3).
y_train_pre = to_categorical(y_train_pre,num_classes=3)
y_test_pre = to_categorical(y_test_pre,num_classes=3)

In [ ]:
print(y_train_pre.shape, y_test_pre.shape)
print(X_train_pre.shape, X_test_pre.shape)

(2037, 3) (227, 3)
(2037,) (227,)


In [ ]:
# create model using the Sequential API
model_use = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dropout(0.5),
    layers.Dense(512, activation="relu"),
    layers.Dropout(0.25),
    layers.Dense(512, activation="relu"),
    layers.Dense(3, activation="softmax")
])

# Compile
model_use.compile(loss= "categorical_crossentropy",
                  optimizer=tf.keras.optimizers.Adam(0.0068),
                  metrics=["accuracy"])
model_use.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE_encoder (KerasLayer)    (None, 512)               256797824 
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 dense_3 (Dense)             (None, 512)               262656    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dense_5 (Dense)             (None, 3)                 1539      
                                                                 
Total params: 257,324,675
Trainable params: 526,851
Non-

In [ ]:
# train the model with USE embedding
model_history_use = model_use.fit(X_train_pre, y_train_pre,
                                  validation_data=(X_test_pre, y_test_pre),
                                  epochs=12,
                                  callbacks=[create_tensorboard_callback(SAVE_DIR,"model_use")])

Saving TensorBoard log files to: model_logs/model_use/20230527-161112
Epoch 1/12
64/64 [==============================] - 7s 25ms/step - loss: 0.6360 - accuracy: 0.7364 - val_loss: 0.4810 - val_accuracy: 0.8194
Epoch 2/12
64/64 [==============================] - 1s 15ms/step - loss: 0.4816 - accuracy: 0.7997 - val_loss: 0.4950 - val_accuracy: 0.8326
Epoch 3/12
64/64 [==============================] - 1s 15ms/step - loss: 0.4417 - accuracy: 0.8223 - val_loss: 0.4265 - val_accuracy: 0.8546
Epoch 4/12
64/64 [==============================] - 1s 15ms/step - loss: 0.3986 - accuracy: 0.8326 - val_loss: 0.4413 - val_accuracy: 0.8370
Epoch 5/12
64/64 [==============================] - 1s 15ms/step - loss: 0.3462 - accuracy: 0.8596 - val_loss: 0.3896 - val_accuracy: 0.8634
Epoch 6/12
64/64 [==============================] - 1s 16ms/step - loss: 0.3195 - accuracy: 0.8684 - val_loss: 0.4531 - val_accuracy: 0.8282
Epoch 7/12
64/64 [==============================] - 1s 16ms/step - loss: 0.3156 - ac

In [ ]:
# predict 
model_5_preds_probs = model_use.predict(X_test_pre)
# convert model prediction probabilities to label format
model_5_preds = tf.squeeze(tf.round(model_5_preds_probs))
# calculate model_1 results
model_5 = calculate_results(y_test_pre, model_5_preds)

8/8 [==============================] - 0s 10ms/step


# Model Performance Comparison ☑

In [ ]:
all_model_results = pd.DataFrame({"Dense":model_1,
                                  "LSTM":model_2,
                                  "GRU":model_3,
                                  "Bidirectional LSTM":model_4,
                                  "Pre-trained":model_5}).T
all_model_results["accuracy"] = round(all_model_results["accuracy"] / 100,3)
all_model_results.sort_values(by="accuracy",ascending=False)

,accuracy,precision,recall,f1
Pre-trained,0.815,0.862,0.815,0.837
LSTM,0.744,0.803,0.744,0.764
GRU,0.722,0.813,0.722,0.762
Dense,0.718,0.813,0.718,0.760
Bidirectional LSTM,0.705,0.803,0.705,0.747


# Model Testing 🧪

In [ ]:
# Define the pre-processing function
def pre_process2(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove punctuation
    punctuation = string.punctuation.replace("'", "")
    text = text.translate(str.maketrans('', '', punctuation))

    # Remove $
    punctuation = string.punctuation.replace("$", "")
    text = text.translate(str.maketrans('', '', punctuation))

    # Remove stop words
    nltk.download('stopwords')
    stop_words = set(stopwords.words('english'))
    text = " ".join(word for word in text.split() if word not in stop_words)

    return text

# Function to make prediction on new text
def predict_sentiment(text):
    # Preprocess the text
    processed_text = pre_process2(text)

    # Tokenize the preprocessed text
    text_seq = tokenizer.texts_to_sequences([processed_text])

    # Pad the sequence
    text_seq_padded = pad_sequences(text_seq, maxlen=max_length)

    # Make prediction
    prediction = model.predict(text_seq_padded)[0]

    return prediction

# Get user input
text_input = input("Enter the text: ")

# Make prediction
prediction = predict_sentiment(text_input)
print("Sentiment Prediction:", prediction)

Enter the text: "However, the Euro 7 proposal is simply not the right way to do this, as it would have an extremely low environmental impact at an extremely high cost."
1/1 [==============================] - 0s 80ms/step
Sentiment Prediction: [0.32501534 0.06486119 0.61012346]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Function to make prediction on new text
def predict_sentiment2(text):
    
    # Make prediction
    prediction = tf.squeeze(model_use.predict([text]))
    
    return prediction

# Get user input
text_input = input("Enter the text: ")

# Make prediction
prediction = predict_sentiment2(text_input)
print("Sentiment Prediction:", prediction)

# Save Model

In [ ]:
# Save Model & Pipeline
import joblib
model_file = open("sentiment_classifier_model.pkl","wb")
joblib.dump(model, model_file)
model_file.close()

In [ ]:
import joblib
import tensorflow as tf
import tensorflow_hub as hub

# Define the custom layer
class USEEncoderLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(USEEncoderLayer, self).__init__(**kwargs)
        self.use_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],  # check the important notes
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE_encoder")

    def call(self, inputs, **kwargs):
        return self.use_layer(inputs)

# Register the custom layer
custom_objects = {"USEEncoderLayer": USEEncoderLayer, "KerasLayer": hub.KerasLayer}

# Load the model with custom layer
with tf.keras.utils.custom_object_scope(custom_objects):
    model = joblib.load("/content/model_use2.pkl")

In [ ]:
# Function to make prediction on new text
def predict_sentiment2(text):
    
    # Make prediction
    prediction = tf.squeeze(model.predict([text]))
    
    return prediction

# Get user input
text_input = input("Enter the text: ")

# Make prediction
prediction = predict_sentiment2(text_input)
print("Sentiment Prediction:", prediction)

Enter the text: "However, the Euro 7 proposal is simply not the right way to do this, as it would have an extremely low environmental impact at an extremely high cost."
1/1 [==============================] - 1s 536ms/step
Sentiment Prediction: tf.Tensor([9.9942374e-01 1.5279527e-04 4.2358576e-04], shape=(3,), dtype=float32)
